# Обучение FastText

В бейзлайне предполагается использовать векторы из библиотеки FastText, которые в данном ноутбуке обучатся на домен чеков

In [1]:
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np

Выгрузим все чеки из размеченного датасета для обучения, а также из тестового датасета

In [2]:
train_df = pd.read_csv("data/train_supervised_dataset.csv").fillna("")
uns_train_df = pd.read_csv("data/train_unsupervised_dataset.csv").fillna("")
test_df = pd.read_csv("data/test_dataset.csv")
names = pd.concat((train_df[["name"]], test_df, uns_train_df[["name"]])).reset_index(drop=True)
names

,name,id
0,Petmax Бантик леопард с красн розой 2шт,NaN
1,87191 Бусы для елки шарики_87191,NaN
2,Футболка Piazza Italia WR011446881,NaN
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN
4,Одежда (вес) 1500,NaN
...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN


In [3]:
train_df['good'].value_counts(1)

               0.03276
брюки          0.01356
пиво           0.01308
вода           0.01232
печенье        0.01164
                ...   
молитвослов    0.00004
медведь        0.00004
супница        0.00004
овчина         0.00004
поддон         0.00004
Name: good, Length: 2820, dtype: float64

In [4]:
train_df['brand'].value_counts(1)

                        0.34020
яшкино                  0.00480
агуша                   0.00364
коровка из кореновки    0.00272
махеевъ                 0.00252
                         ...   
пласти лаб              0.00004
феррогематоген          0.00004
cbr                     0.00004
helcom                  0.00004
smail                   0.00004
Name: brand, Length: 6976, dtype: float64

In [103]:
import re

from cyrtranslit import to_cyrillic, to_latin
lat_to_cyr = str.maketrans("aekmhopctyx", "аекмнорстух")
cyr_to_lat = str.maketrans("аекмнорстух", "aekmhopctyx")
stop_words = set(['г', 'кг', 'шт', 'мл', 'л', 'литр', 'мг', 'гр', 'км', 'мм', 'mm', 'уп'])

def replace_camel_case(s):
    matches = len(re.findall(r'(?<=[a-zа-я])([A-ZА-Я])', s))
    if matches > 2:
        return re.sub(r'(?<=[a-zа-я])([A-ZА-Я])', r' \1', s)
    else:
        return s
    
def split_on_language_change(s):
    s = re.sub(r'(?<=[a-zа-я])(?=[A-ZА-Я0-9])', r' ', s)
    s = re.sub(r'(?<=[A-ZА-Я0-9])(?=[a-zа-я])', r' ', s)
    return s


def insert_space_after_one(s):
    return re.sub(r'(1)(?=[A-Za-zА-Яа-я])', r'\1 ', s)

def replace_zero(s):
    s = re.sub(r'(?<=[A-Za-z])0(?=[A-Za-z])', 'o', s)
    s = re.sub(r'(?<=[А-Яа-я])0(?=[А-Яа-я])', 'о', s)
    return s

def preprocess_text(text):
    
    text = replace_zero(text)
    
    text = re.sub('\d+', '1', text)  # replace numbers to 1
    text = replace_camel_case(text)
    text = re.sub('д/', 'для ', text)
    text = re.sub('Д/', 'для ', text)
    text = insert_space_after_one(text)
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\w\s]', ' ', text)  # remove punctuation
    words = []
    for w in text.lower().split():
#         if len(w) < 2:
#             continue
        
        num_eng_chars = len(re.findall(r'[a-z]', w))
        num_ru_chars = len(re.findall(r'[а-я]', w))
        if num_eng_chars and num_ru_chars:
            if num_eng_chars > num_ru_chars:
                w = w.translate(cyr_to_lat)
            else:
                w = w.translate(lat_to_cyr)
        
        if w in stop_words:
            continue
        
        # если нет транзиторов
        w = split_on_language_change(w)
        if ' ' in w:
            words.extend(w.split())
        else:
            if w == 'нести':
                w = 'nestea'
            if w == 'эпика':
                w = 'epica'
            if w == 'хелен':
                w = 'helen'
            if w == 'харпер':
                w = 'harper'
            if w == 'тимотей':
                w = 'timotei'
            if w == 'тесс':
                w = 'tess'
            if w == 'кроненбург':
                w = 'kronenbourg'
            if w == 'пай':
                w = 'pie'
            if w == 'чоко':
                w = 'choko'
            if w == 'салтон':
                w = 'salton'
            words.append(w)
        
    return words

In [88]:
text = train_df['name'].sample().values[0]
text

'Козинак из воздушных зерен риса  Тимоша 50г'

In [89]:
train_df[train_df['name'] == text]

,id,name,good,brand
4670,4670,Козинак из воздушных зерен риса Тимоша 50г,козинак,тимоша


In [194]:
%%timeit
preprocess_text(text)

31.1 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [104]:
good_founded_list = []
brand_founded_list = []
DEBUG = True

for i, r in train_df.iterrows():
    tokens = preprocess_text(r['name'])
    brand = r['brand']
    good = r['good']
    if len(re.findall(r'[a-z]', brand)) > 0 and brand not in tokens:
        DEBUG = False
    else:
        DEBUG = False
    if DEBUG:
        print(r['name'])
    if brand:
        if DEBUG:
            print(f"brand: {brand}")
        brand_founded_list.append(brand in tokens)
    if good:
        if DEBUG:
            print(f"good: {good}")
        good_founded_list.append(good in tokens)
    if DEBUG:
        print(tokens)
        print(f'\n\n')

In [105]:
# baseline:
#     good:  0.849
#     brand: 0.57
        
# hard_preprocc:
#     good:  0.919689
#     brand: 0.694089

# hard_preprocc + brand renaming
#     good:  0.919689
#     brand: 0.695180

In [106]:
np.mean(good_founded_list)

0.9196890120342418

In [107]:
np.mean(brand_founded_list)

0.6951803576841468

In [15]:
to_cyrillic('epica')

'епица'

In [11]:
preprocess_text(text)

['декоративная',
 'клейкая',
 'лента',
 'металлик',
 'намотка',
 '1',
 'м',
 'ширина',
 '1',
 '1',
 'см',
 '1',
 'в',
 'банке',
 'цена',
 'за',
 'банку']

In [12]:
%%time
names["tokens"] = names["name"].apply(preprocess_text)
names

CPU times: user 57.7 s, sys: 761 ms, total: 58.5 s
Wall time: 58.5 s


,name,id,tokens
0,Petmax Бантик леопард с красн розой 2шт,NaN,"[petmax, бантик, леопард, с, красн, розой, 1]"
1,87191 Бусы для елки шарики_87191,NaN,"[1, бусы, для, елки, шарики_1]"
2,Футболка Piazza Italia WR011446881,NaN,"[футболка, piazza, italia, wr, 1]"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN,"[1, yi, 1, 1, x, one, заколка, для, волос, для..."
4,Одежда (вес) 1500,NaN,"[одежда, вес, 1]"
...,...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN,"[f, 1, спонж, для, макияжа, фигурный]"
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN,"[1, 1, 1, 1, дифф, arom, 1]"
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN,"[матрас, надувной, 1, 1, см, tropical, bird, з..."
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN,"[пододеяльник, стм, страйп, 1, сп, размер, 1, ..."


In [262]:
names.to_csv('data/preprocc_names.csv', index=False)

Обученные векторы положим в файл `fasttext.model` для использования в моделях

In [13]:
%%time
FastText(names["tokens"], vector_size=256, window=5, min_count=5, epochs=30).save("fasttext_models/fasttext_256_hardpreprocc_fitted.model")

CPU times: user 28min 26s, sys: 41.1 s, total: 29min 7s
Wall time: 10min 12s
